# PREDICCIÓN DE VENTAS SEMANALES DE WALMART

### Modelo de regresión con Pycaret y Sklearn

#### Etapas del modelo
1. Análisis de calidad de datos
2. Análisis exploratorio de datos (EDA)
3. Ingenieria de caracteristicas (creación de columnas, categorias, eliminar, limpiezas,transformaciones, etc)
4. Fase de modelado:
- Pycaret
- Sklearn
5. Comparación de modelos con métricas
6. Resultados y conclusiones


## Fase 1:
### Instalación de librerías y carga de datos desde MySQL

In [ ]:
#pip install mysql

  Using cached mysql-0.0.3-py3-none-any.whl (1.2 kB)
  Using cached mysqlclient-2.2.7-cp311-cp311-win_amd64.whl (207 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#pip install mysql-connector-python

  Using cached mysql_connector_python-9.5.0-cp311-cp311-win_amd64.whl (16.5 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#pip install pandas

In [ ]:
#pip install numpy
 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#pip install seaborn 

  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached matplotlib-3.10.7-cp311-cp311-win_amd64.whl (8.1 MB)
  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl (225 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ---------------------------------------- 2.3/2.3 MB 29.7 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.9-cp311-cp311-win_amd64.whl (73 kB)
  Using cached pillow-12.0.0-cp311-cp311-win_amd64.whl (7.0 MB)
  Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl (8.9 MB)
     --------------------------------------- 38.7/38.7 MB 34.4 MB/s eta 0:00:00
  Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Importación de librerías

## Manipulación de datos
import pandas as pd
import numpy as np

# Visualización de datos
import seaborn as sns
import matplotlib.pyplot as plt

# Modelado y evaluación
import sklearn 

In [9]:

# Procesamiento y modelado con skylearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Modelos robustos de regresión
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR

from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [14]:
# Conectar base de datos con MySQL
import mysql.connector
from mysql.connector import Error

Conexion = mysql.connector.connect(
    host='localhost',       
    database='curso_python',
    user='root',
    password='Maracuya134',
    port='3306'
)

    

In [15]:
Query = "SELECT * FROM curso_python.walmart;"

In [16]:
df_walmart = pd.read_sql(Query, Conexion)

C:\Users\rrosm\AppData\Local\Temp\ipykernel_19064\156267084.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_walmart = pd.read_sql(Query, Conexion)


In [17]:
# Analisis exploratorio de datos
df_walmart.head()
df_walmart.info()
df_walmart.describe()
df_walmart.isnull().sum()
df_walmart.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


np.int64(0)

In [18]:
df_walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.57,211.10,8.11
1,1,12-02-2010,1641957.44,1,38.51,2.55,211.24,8.11
2,1,19-02-2010,1611968.17,0,39.93,2.51,211.29,8.11
3,1,26-02-2010,1409727.59,0,46.63,2.56,211.32,8.11
4,1,05-03-2010,1554806.68,0,46.50,2.62,211.35,8.11


In [19]:
df_walmart.shape

(6435, 8)

### Analisis de calidad de datos

In [20]:
# Verificación de valores nulos
nulos = df_walmart.isnull().sum()
nulos

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [21]:
# Verificación de duplicados
duplicados = df_walmart.duplicated().sum()
duplicados

np.int64(0)

In [22]:
# Conversión de la columna 'Date' a tipo datetime
df_walmart['Date'] = pd.to_datetime(df_walmart['Date'], format='%d-%m-%Y')

# Identificar fecha minima y máxima
fecha_min = df_walmart['Date'].min()
fecha_max = df_walmart['Date'].max()

print(f"Fecha mínima: {fecha_min}")
print(f"Fecha máxima: {fecha_max}")

Fecha mínima: 2010-02-05 00:00:00
Fecha máxima: 2012-10-26 00:00:00


In [23]:
# Estadísticas basicas
df_walmart.describe()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.00,6435,6435.00,6435.00,6435.00,6435.00,6435.00,6435.00
mean,23.00,2011-06-17 00:00:00,1046964.88,0.07,60.66,3.36,171.58,8.00
min,1.00,2010-02-05 00:00:00,209986.25,0.00,-2.06,2.47,126.06,3.88
25%,12.00,2010-10-08 00:00:00,553350.10,0.00,47.46,2.93,131.74,6.89
50%,23.00,2011-06-17 00:00:00,960746.04,0.00,62.67,3.44,182.62,7.87
75%,34.00,2012-02-24 00:00:00,1420158.66,0.00,74.94,3.73,212.74,8.62
max,45.00,2012-10-26 00:00:00,3818686.45,1.00,100.14,4.47,227.23,14.31
std,12.99,NaN,564366.62,0.26,18.44,0.46,39.36,1.88


In [24]:
df_walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.57,211.10,8.11
1,1,2010-02-12,1641957.44,1,38.51,2.55,211.24,8.11
2,1,2010-02-19,1611968.17,0,39.93,2.51,211.29,8.11
3,1,2010-02-26,1409727.59,0,46.63,2.56,211.32,8.11
4,1,2010-03-05,1554806.68,0,46.50,2.62,211.35,8.11
